# TFIDF

### Libraries

In [3]:
!pip install youtokentome

In [2]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import youtokentome as yttm

### Download Data

In [2]:
! wget https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml -P ./data
! wget https://github.com/language-ml/2-LM-embedding-projects/raw/main/problem3/doc_collection.zip -P ./data
! unzip ./data/doc_collection.zip -d ./data

### Read data

set path of data in `PATH` variable

In [4]:
PATH = './data/IR_dataset/'
PATH = PATH.rstrip('/')

store txt files into a list named `doc`

In [5]:
docs = []
for index in range(0, 3258):
    with open(f"{PATH}/{index}.txt", 'r', encoding='utf8') as file_reader:
        docs.append(file_reader.read())

show example of data

In [6]:
for doc in docs[:3]:
    print (doc[:500])
    print('-' * 50)

برخی از هواداران مصدق یا اعضای جبهه ملی که در زمان نخست‌وزیری مصدق، از جبهه ملی یا از هیئت‌وزیران کنار گذاشته شده یا کنار رفتند، پس از جدایی از مصدق، به انتقاد از کارنامه وی پرداختند و حتی برای سرنگونی‌اش تلاش کردند. برخی از این افراد عبارت‌اند از:
* فضل‌الله زاهدی (نخست‌وزیر کودتا)
* علی امینی
* حسین مکی، (که در آغاز سرباز فداکار وطن نامیده شد ولی در پایان به دلیل مخالفت با مصدق از سوی هواداران جبهه ملی سرباز خطاکار وطن خطاب می‌شد)
* مظفر بقایی (به دلیل اتهام مشارکت در قتل سرتیپ افشار طوس و سپس
--------------------------------------------------
جبهه ملی ایران که به اختصار جبهه ملی نیز خوانده می‌شود سازمان سیاسی ملی‌گرای سکولار، دموکرات و جمهوری‌خواه فعال در ایران است، که در حال حاضر تحت رهبری دکتر موسویان فعالیت می‌کند.

جبهه ملی ایران در سال ۱۳۲۸ توسط سیاستمدارانی از قبیل محمد مصدق، حسین فاطمی و کریم سنجابی تأسیس شد و به پیش نهاد حسین فاطمی، ملی شدن صنعت نفت ایران را مطرح کرد. دکتر مصدق در کتاب خاطرات خود می‌گوید: «پیشنهاد ملی‌شدن صنعت نفت در سراسر کشور ابتکار شادروان دکتر حسین فاطمی

In [5]:
### data clean

### bpe

In [7]:
BPE_PATH = './bpe/'
BPE_PATH = BPE_PATH.rstrip('/')
os.makedirs(BPE_PATH, exist_ok=True)
train_data_path = BPE_PATH + "/train_data.txt"
model_path = BPE_PATH + "/bpe.model"

- Train

In [ ]:
# Generating corpus file with training data
pretrained_bpe_texts = ' '.join(docs)

with open(train_data_path, "w") as w_file:
    w_file.write(pretrained_bpe_texts)

# Training model
yttm.BPE.train(data=train_data_path, vocab_size=5000, model=model_path, unk_id=1, bos_id=2, eos_id=3)

- Load
- Test

In [6]:
# Loading model
bpe = yttm.BPE(model=model_path)

# first document as a test 
test_text = docs[0]

# Two types of tokenization
print(bpe.encode(test_text, output_type=yttm.OutputType.SUBWORD))

### TFIDF TOP K Relevent Documents 

In [ ]:
## function: tokenizer persian 

In [ ]:
## list: stop word persian

In [15]:
class tfidf:
    
    def __init__(self, docs, ngram_range, bpe_model=None):
        self._min_df = 1
        self._max_df=0.5
        self._max_features=10000
        self._docs = docs
        self._bpe = bpe_model
        self._ngram_range = ngram_range
        if self._bpe:
            self._model_tfidf = TfidfVectorizer(analyzer="word", min_df=self._min_df, max_df=self._max_df, max_features=self._max_features, ngram_range=self._ngram_range, tokenizer= lambda x: self.bpe_tokenizer(x))
        else:
            self._model_tfidf = TfidfVectorizer(analyzer="word", min_df=self._min_df, max_df=self._max_df, max_features=self._max_features, ngram_range=self._ngram_range)

        self._matrix = self._model_tfidf.fit_transform(docs)

    def bpe_tokenizer(self, text):
        tokens = self._bpe.encode(text, output_type=yttm.OutputType.SUBWORD)
        return tokens 
        

    def tfidf_top_k(self, query, k=2):
        
        query_tfidf = self._model_tfidf.transform([query])

        # Stores cosine similarity scores
        doc_scores = []

        # Compute the cosine similarity scores
        for doc in self._matrix:
            doc_scores.append(cosine_similarity(query_tfidf, doc)[0][0])

        # Sort list of doc_scores and return the top k indices of highest scores
        sorted_scores = sorted(enumerate(doc_scores), key=lambda ind_score: ind_score[1], reverse=True)
        top_doc_indices = [ind for ind, score in sorted_scores[:k]]

        return top_doc_indices

In [1]:
query = "آدولف هیتلر شکست و مرگ"

###  Word Level: ngram (1,1)

In [25]:
tf_obj = tfidf(docs=docs, ngram_range=(1,1), bpe_model=None)
tf_obj.tfidf_top_k(query, 20)

[484,
 482,
 349,
 343,
 345,
 344,
 466,
 481,
 341,
 347,
 1544,
 463,
 342,
 443,
 469,
 442,
 483,
 220,
 354,
 449]

###  Word Level: ngram (1,2)

In [26]:
tf_obj = tfidf(docs=docs, ngram_range=(1,2), bpe_model=None)
tf_obj.tfidf_top_k(query, 20)

[484,
 482,
 344,
 345,
 343,
 349,
 481,
 341,
 466,
 1544,
 342,
 347,
 463,
 354,
 220,
 443,
 469,
 483,
 442,
 449]

### BPE: ngram (1,1)

In [27]:
tf_obj = tfidf(docs=docs, ngram_range=(1,1), bpe_model=bpe)
tf_obj.tfidf_top_k(query, 20)

[484,
 482,
 344,
 345,
 343,
 347,
 342,
 341,
 466,
 481,
 442,
 1544,
 443,
 463,
 220,
 2080,
 483,
 469,
 349,
 449]

### BPE: ngram (1,2)

In [28]:
tf_obj = tfidf(docs=docs, ngram_range=(1,2), bpe_model=bpe)
tf_obj.tfidf_top_k(query, 20)

[484,
 482,
 344,
 345,
 343,
 341,
 466,
 347,
 481,
 342,
 1544,
 463,
 2080,
 442,
 220,
 469,
 483,
 354,
 443,
 349]